In [ ]:
import math as math
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from scipy.stats import chi2
import scipy
from IPython.display import HTML, display
import tabulate

Функций подсчета элементов в указанном промежутке

In [ ]:
# [start, end)
def elementInRange(elements, start, end):
    count = 0
    for el in elements:
        if (el >= start) and (el < end):
            count += 1
    return count

Функций создания интервальных рядов

In [ ]:
def makeIntervals(samples, start, delt, count):
    samples.sort()
    intervals = []
    for i in range(1, count + 1):
        currentStart = start + (i - 1) * delt
        currentEnd = start + i * delt
        intervals.append([(currentStart + currentEnd) / 2,
                          elementInRange(samples, currentStart, currentEnd),
                          (currentStart, currentEnd)])
    intervals[-1][1] += 1
    return intervals

Создадим выборки

In [ ]:
def next_element(previous, mu): #функция подсчета следующего x
    return previous * mu - int(previous * mu)

In [ ]:
size = 100
m = 2347 #простое число
uniform_selection = [0.99]
for element in range(1, size):
    uniform_selection.append(next_element(uniform_selection[element - 1], m))  

Экспоненциальное распределение

In [ ]:
lambda_parameter = 10
exponential_selection = []
for element in range(size):
    exponential_selection.append(-(1/lambda_parameter)*math.log(1-uniform_selection[element]))

In [ ]:
plt.hist(exponential_selection, 50, color='r')
plt.show() 

In [ ]:
exponential_selection.sort()
exp_range = exponential_selection[-1] - exponential_selection[0]
print("Размах: ", exp_range)

In [ ]:
number_intervals = 5

In [ ]:
exp_delta = exp_range / number_intervals
print("Delta = ", exp_delta)

In [ ]:
exp_intervals = makeIntervals(exponential_selection, exponential_selection[0], exp_delta, number_intervals)
exp_tuples = []
exp_avg = []
exp_count = []
for interval in exp_intervals:
    exp_avg.append(interval[0])
    exp_count.append(interval[1])
    exp_tuples.append(interval[2])

In [ ]:
table = []
table.append([exp_tuples, exp_avg, exp_count])
display(HTML(tabulate.tabulate({"Интервал": exp_tuples, "Количество элементов":exp_count, "Среднее значение в интервале": exp_avg}, tablefmt='html', headers="keys")))

In [ ]:
exp_hash = {"niui": [],
         "niui_2": []}
false_zero = exponential_selection[49]
local_estim_table_exp = PrettyTable(["Ji", "xi*", "ni", "ui", "ni*ui", "ni*ui^2", "ni(ui + 1)^2"])
for interval in exp_intervals:
    ui = (interval[0] - false_zero) / exp_delta
    ni_ui = interval[1] * ui
    ni_ui_sqr = interval[1] * (ui * ui)
    control = interval[1] * pow((ui + 1), 2)
    exp_hash["niui"].append(ni_ui)
    exp_hash["niui_2"].append(ni_ui_sqr) 
    local_estim_table_exp.add_row([interval[2], interval[0], interval[1], ui, ni_ui, ni_ui_sqr, control])

In [ ]:
display(HTML(local_estim_table_exp.get_html_string()))

In [ ]:
exp_moment1 = sum(exp_hash["niui"]) / size
exp_moment2 = sum(exp_hash["niui_2"]) / size

average_chosen = exp_delta * exp_moment1 + false_zero
chosen_dispersion = (exp_moment2 - pow(exp_moment1, 2)) * pow(exp_delta, 2)

# Local points estimation
est_lambda = 1 / average_chosen
est_lambda_sqr = 1 / chosen_dispersion

In [ ]:
print("Среднее выборочное: ", average_chosen, "\nЛямбда (ескп. закон): ", est_lambda,
      "\nЛямбда квадрат: ", est_lambda_sqr)

In [ ]:
alpha_chi_coef = 0.05
x_alpha = chi2.ppf(alpha_chi_coef, size * 2)
x_beta = chi2.ppf(1 - alpha_chi_coef, size * 2)
gamma = 1 - alpha_chi_coef * 2  # alpha == beta
lambda_start = x_alpha / (2 * size * average_chosen)
lambda_end = x_beta / (2 * size * average_chosen)

In [ ]:
print("Границы доверительного интервала для лямбда: ", lambda_start, " ", lambda_end, "\nГамма: ", gamma)

In [ ]:
hyp_table_exp = PrettyTable(["Ji", "ni", "n'i", "ni-n'i", "(ni-n'i)^2", "((ni-n'i)^2)/n'i"])
chi_sample = 0
for interval in exp_intervals:
    n_i = size * (math.exp(-1 * est_lambda * interval[2][0]) - math.exp(-1 * est_lambda * interval[2][1]))
    ni_sub = interval[1] - n_i
    ni_sub_sqr = pow(interval[1] - n_i, 2)
    control = ni_sub_sqr / n_i
    chi_sample += control
    hyp_table_exp.add_row([interval[2], interval[1], n_i, ni_sub, ni_sub_sqr, control])

In [ ]:
display(HTML(hyp_table_exp.get_html_string()))

In [ ]:
probability = 0.9

In [ ]:
chi_critical = chi2.ppf(probability, number_intervals - 2)

In [ ]:
print("Хи^2 кр: ", chi_critical, "Хи^2 выб: ", chi_sample)
if chi_critical > chi_sample:
    print("Гипотеза принята на уровне значимости альфа:", 1-probability)
else:
    print("Гипотеза отвержена")

Нормальное распределение

In [ ]:
m = 5
sigma = 2
normal_selection = [math.sqrt(-2*math.log(uniform_selection[0]))*math.cos(2*math.pi*uniform_selection[0])]
for element in range(1, size):
    normal_selection.append(math.sqrt(-2*math.log(uniform_selection[element-1]))*math.cos(2*math.pi*uniform_selection[element])) 
for element in range(size):
    normal_selection[element] = m + sigma*normal_selection[element]

In [ ]:
plt.hist(normal_selection, 50)
plt.show()

In [ ]:
normal_selection.sort()
norm_range = normal_selection[-1] - normal_selection[0]

In [ ]:
print("Размах: ", norm_range)

In [ ]:
number_intervals = 5

In [ ]:
norm_delta = norm_range / number_intervals

In [ ]:
print("Delta = ", norm_delta)

In [ ]:
normIntervals = makeIntervals(normal_selection, normal_selection[0], norm_delta, number_intervals)
norm_tuples = []
norm_avg = []
norm_count = []
for interval in exp_intervals:
    norm_avg.append(interval[0])
    norm_count.append(interval[1])
    norm_tuples.append(interval[2])

In [ ]:
table = []
table.append([exp_tuples, exp_avg, exp_count])
display(HTML(tabulate.tabulate({"Интервал": exp_tuples, "Количество элементов":exp_count, "Среднее значение в интервале": exp_avg}, tablefmt='html', headers="keys")))

In [ ]:
local_estim_table_norm = PrettyTable(["xi*", "ni", "ui", "ni*ui", "ni*ui^2",
                                 "ni(ui + 1)^2", "ni*ui^3", "ni*ui^4", "ni*(ui+1)^4"])
false_zero = normal_selection[49]
momentum_hash = {"momentum_1": [],
                    "momentum_2": [],
                    "momentum_3": [],
                    "momentum_4": [], }
for interval in normIntervals:
    ui = (interval[0] - false_zero) / norm_delta
    ni_ui = interval[1] * ui
    ni_ui_sqr = interval[1] * ui * ui
    control_1 = interval[1] * pow((ui + 1), 2)
    ni_ui_cube = interval[1] * pow(ui, 3)
    ni_ui_4 = interval[1] * pow(ui, 4)
    control_2 = interval[1] * pow((ui + 1), 4)
    momentum_hash["momentum_1"].append(ni_ui)
    momentum_hash["momentum_2"].append(ni_ui_sqr)
    momentum_hash["momentum_3"].append(ni_ui_cube)
    momentum_hash["momentum_4"].append(ni_ui_4) 
    local_estim_table_norm.add_row([interval[0], interval[1], ui, ni_ui, ni_ui_sqr, control_1, ni_ui_cube, ni_ui_4, control_2])

In [ ]:
display(HTML(local_estim_table_norm.get_html_string()))

In [ ]:
moment_1 = (sum(momentum_hash["momentum_1"])) / size
moment_2 = (sum(momentum_hash["momentum_2"])) / size
moment_3 = (sum(momentum_hash["momentum_3"])) / size
moment_4 = (sum(momentum_hash["momentum_4"])) / size

In [ ]:
expected_value_estimation = norm_delta * moment_1 + false_zero
expected_disp_estimation = (moment_2 - pow(moment_1, 2)) * pow(norm_delta, 2)
expected_deviation = pow(expected_disp_estimation, 0.5)
centMom_3 = (moment_3 - 3 * moment_1 * moment_2 + 2 * pow(moment_1, 3)) * pow(norm_delta, 3)
centMom_4 = (moment_4 - 4 * moment_1 * moment_3 + pow(moment_1, 2) * 6 * moment_2 - 3 * pow(moment_1, 4)) * pow(
    norm_delta, 4)

asymmetry = centMom_3 / pow(expected_deviation, 3)
excess = centMom_4 / pow(expected_deviation, 4) - 3

In [ ]:
print("Оценка мат ожидания(среднее выборочное): ", expected_value_estimation, "\nВыборочная дисперсия: ",
      expected_disp_estimation,
      "\nОценка отклонения: ", expected_deviation, "\nАсимметрия: ", asymmetry, "Эксцесс", excess)


In [ ]:
t_coef = 1.66

In [ ]:
temp = 0  # temporary var for sum to get X of set (среднее выборочное)
for x in normal_selection:
    temp += pow((x - expected_value_estimation), 2)
fixed_chosen_disp = temp / (size - 1)
m_start = expected_value_estimation - (pow(fixed_chosen_disp / size, 0.5)) * t_coef
m_end = expected_value_estimation + (pow(fixed_chosen_disp / size, 0.5)) * t_coef

In [ ]:
alpha_chi_coef = 0.05
x_alpha = chi2.ppf(alpha_chi_coef, size - 1)
x_beta = chi2.ppf(1 - alpha_chi_coef, size - 1)
deviation_start = pow((fixed_chosen_disp * (size - 1)) / x_beta, 0.5)
deviation_end = pow((fixed_chosen_disp * (size - 1)) / x_alpha, 0.5)

In [ ]:
print("Интервальная оценка для нормального m: ", m_start, m_end,
      "\nИнтервальная оценка для sigma:", deviation_start, deviation_end)

In [ ]:
hyp_table_norm = PrettyTable(["xi", "xi+1", "xi - x_ch", "xi+1 - x_ch",
                                   "Zi = (xi-x_ch)/sigma", "Zi+1 = ((xi+1)-x_ch)/sigma"])
chosen_sigma = pow(expected_disp_estimation, 0.5)
z_array = []
for interval in normIntervals:
    x_ch = expected_value_estimation
    xi = interval[2][0]
    xi_1 = interval[2][1]
    xi_sub_xch = xi - x_ch
    xi_1_sub_xch = xi_1 - x_ch
    zi = xi_sub_xch / chosen_sigma
    zi_1 = xi_1_sub_xch / chosen_sigma
    z_array.append([(zi, zi_1), interval[1]])
    hyp_table_norm.add_row([xi, xi_1, xi_sub_xch, xi_1_sub_xch, zi, zi_1])

In [ ]:
display(HTML(hyp_table_norm.get_html_string()))

In [ ]:
z_hyp_table_norm = PrettyTable(["zi", "zi+1", "Fo(zi)", "Fo(zi+1)", "Pi = Fo(zi) - Fo(zi+1)", "nPi = ni'"])
chi_hyp_table_norm = PrettyTable(["ni", "ni'", "(ni-ni')^2", "((ni-ni')^2)/ni'"])
x_norm = 0
for i in z_array:
    zi = i[0][0]
    zi_1 = i[0][1]
    fi = scipy.stats.norm.cdf(zi) - 0.5
    fi_1 = scipy.stats.norm.cdf(zi_1) - 0.5
    pi = abs(fi - fi_1)
    n_i = size * pi
    x_norm += (pow(i[1] - n_i, 2)) / n_i
    z_hyp_table_norm.add_row([zi, zi_1, fi, fi_1, pi, n_i])
    chi_hyp_table_norm.add_row([i[1], n_i, pow(i[1] - n_i, 2), (pow(i[1] - n_i, 2)) / n_i])

In [ ]:
display(HTML(z_hyp_table_norm.get_html_string()))

In [ ]:
display(HTML(chi_hyp_table_norm.get_html_string()))

In [ ]:
probability = 0.9
chi_critical = chi2.ppf(probability, number_intervals - 3)

In [ ]:
print("Хи^2 кр: ", chi_critical, "Хи^2 выб: ", x_norm)
if x_norm < chi_critical:
    print("Гипотеза принята на уровне значимости альфа: ", 1-probability)
else:
    print("Гипотеза отвержена")